^fill^README.md^here^

^fill^theory.md^here^

## Demonstration

### 1. Setup

#### 1.1. Library imports

Import libraries needed by the calculation. The external libraries used are:

- [numpy](http://www.numpy.org/)

- [DataModelDict](https://github.com/usnistgov/DataModelDict)

- [atomman](https://github.com/usnistgov/atomman)

- [iprPy](https://github.com/usnistgov/iprPy)

In [ ]:
# Standard library imports
from pathlib import Path
import os
import sys
import uuid
import shutil
import datetime
from copy import deepcopy

# http://www.numpy.org/
import numpy as np 

# https://github.com/usnistgov/DataModelDict 
from DataModelDict import DataModelDict as DM

# https://github.com/usnistgov/atomman 
import atomman as am
import atomman.lammps as lmp
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy

print('Notebook last executed on', datetime.date.today(), 'using iprPy version', iprPy.__version__)

#### 1.2. Default calculation setup

In [ ]:
# Specify calculation style
calc_style = 'point_defect_static'

# If workingdir is already set, then do nothing (already in correct folder)
try:
    workingdir = workingdir

# Change to workingdir if not already there
except:
    workingdir = Path('calculationfiles', calc_style)
    if not workingdir.is_dir():
        workingdir.mkdir(parents=True)
    os.chdir(workingdir)

### 2. Assign values for the calculation's run parameters

#### 2.1. Specify system-specific paths

- __lammps_command__ (required) is the LAMMPS command to use.

- __mpi_command__ MPI command for running LAMMPS in parallel. A value of None will run simulations serially.

In [ ]:
lammps_command = 'lmp_serial'
mpi_command = None

#### 2.2. Load interatomic potential

- __potential_name__ gives the name of the potential_LAMMPS reference record in the iprPy library to use for the calculation.  

- __potential_file__ gives the path to the potential_LAMMPS reference record to use.  Here, this parameter is automatically generated using potential_name and librarydir.

- __potential_dir__ gives the path for the folder containing the artifacts associated with the potential (i.e. eam.alloy file).  Here, this parameter is automatically generated using potential_name and librarydir.

- __potential__ is an atomman.lammps.Potential object (required).  Here, this parameter is automatically generated from potential_file and potential_dir.

In [ ]:
potential_name = '1999--Mishin-Y--Ni--LAMMPS--ipr1'

# Define potential_file and potential_dir using librarydir and potential_name
potential_file = Path(iprPy.libdir, 'potential_LAMMPS', f'{potential_name}.json')
potential_dir = Path(iprPy.libdir, 'potential_LAMMPS', potential_name)

# Initialize Potential object using potential_file and potential_dir.
potential = lmp.Potential(potential_file, potential_dir)
print('Successfully loaded potential', potential)

#### 2.3. Load initial unit cell system

- __prototype_name__ gives the name of the crystal_prototype reference record in the iprPy library to load. 

- __symbols__ is a list of the potential's elemental model symbols to associate with the unique atom types of the loaded system. 

- __box_parameters__ is a list of the a, b, c lattice constants to assign to the loaded file.

- __load_file__ gives the path to the atomic configuration file to load for the ucell system.  Here, this is generated automatically using prototype_name and librarydir.

- __load_style__ specifies the format of load_file.  Here, this is automatically set for crystal_prototype records.

- __load_options__ specifies any other keyword options for properly loading the load_file.  Here, this is automatically set for crystal_prototype records.

- __ucell__ is an atomman.System representing a fundamental unit cell of the system (required).  Here, this is generated using the load parameters and symbols.

In [ ]:
prototype_name = 'A1--Cu--fcc'
symbols = ['Ni']
box_parameters = uc.set_in_units([3.52, 3.52, 3.52], 'angstrom')

# Define load_file using librarydir and prototype_name
load_file = Path(iprPy.libdir, 'crystal_prototype', f'{prototype_name}.json')

# Define load_style and load_options for crystal_prototype records
load_style = 'system_model'
load_options = {}

# Create ucell by loading prototype record
ucell = am.load(load_style, load_file, symbols=symbols, **load_options)

# Rescale ucell using box_parameters
ucell.box_set(a=box_parameters[0], b=box_parameters[1], c=box_parameters[2], scale=True)

print(ucell)

#### 2.4. Specify the defect parameters

- __pointdefect_name__ gives the name of a point-defect reference record in the iprPy library containing point defect input parameters.

- __pointdefect_file__ gives the path to a point_defect reference containing point defect input parameters.  Here, this is built automatically using pointdefect_name and librarydir.

- __point_kwargs__ (required) is a dictionary or list of dictonaries containing parameters for generating the defect. Here, values are extracted from pointdefect_file. Allowed keywords are:

    - __ptd_type__ indicates which defect type to generate: 'v' for vacancy, 'i' for interstitial, 's' for substitutional, or 'db' for dumbbell.
    
    - __atype__ is the atom type to assign to the defect atom ('i', 's', 'db' ptd_types).
    
    - __pos__ specifies the position for adding the defect atom (all ptd_types).
    
    - __ptd_id__ specifies the id of an atom in the initial system where the defect is to be added. Alternative to using pos ('v', 's', 'db' ptd_types).
    
    - __db_vect__ gives the vector associated with the dumbbell interstitial to generate ('db' ptd_type).
    
    - __scale__ indicates if pos and db_vect are in absolute (False) or box-relative (True) coordinates. Default is False.
    
    - __atol__ is the absolute tolerance for position-based searching. Default is 1e-3 angstroms.


In [ ]:
pointdefect_name = 'A1--Cu--fcc--vacancy'
#pointdefect_name = 'A1--Cu--fcc--1nn-divacancy'
#pointdefect_name = 'A1--Cu--fcc--2nn-divacancy'
#pointdefect_name = 'A1--Cu--fcc--100-dumbbell'
#pointdefect_name = 'A1--Cu--fcc--110-dumbbell'
#pointdefect_name = 'A1--Cu--fcc--111-dumbbell'
#pointdefect_name = 'A1--Cu--fcc--octahedral-interstitial'
#pointdefect_name = 'A1--Cu--fcc--tetrahedral-interstitial'
#pointdefect_name = 'A1--Cu--fcc--crowdion-interstitial'

# Define pointdefect_file using librarydir and pointdefect_name
pointdefect_file = Path(iprPy.libdir, 'point_defect', f'{pointdefect_name}.json')

# Parse pointdefect_file using iprPy.input.interpret()
defectinputs = {'ucell':ucell, 'pointdefect_file':pointdefect_file}
iprPy.input.subset('pointdefect').interpret(defectinputs)

# Extract point_kwargs
point_kwargs = defectinputs['point_kwargs']
print('point_kwargs =')
point_kwargs

#### 2.5. Modify system

- __sizemults__ list of three integers specifying how many times the ucell vectors of $a$, $b$ and $c$ are replicated in creating system.

- __system__ is an atomman.System to perform the scan on (required). 

In [ ]:
sizemults = [10, 10, 10]

# Generate system by supersizing ucell
system = ucell.supersize(*sizemults)
print('# of atoms in system =', system.natoms)

#### 2.6. Specify calculation-specific run parameters

- __energytolerance__ is the energy tolerance to use during the minimizations. This is unitless.

- __forcetolerance__ is the force tolerance to use during the minimizations. This is in energy/length units.

- __maxiterations__ is the maximum number of minimization iterations to use.

- __maxevaluations__ is the maximum number of minimization evaluations to use.

- __maxatommotion__ is the largest distance that an atom is allowed to move during a minimization iteration. This is in length units.

In [ ]:
energytolerance = 1e-8
forcetolerance = uc.set_in_units(0.0, 'eV/angstrom')
maxiterations = 10000
maxevaluations = 100000
maxatommotion = uc.set_in_units(0.01, 'angstrom')

### 3. Define calculation function(s) and generate template LAMMPS script(s)

#### 3.1. min.template

In [ ]:
with open('min.template', 'w') as f:
    f.write("""^fill^min.template^here^""")

#### 3.2. pointdefect()

In [ ]:
^fill^calc_point_defect_static.py(pointdefect)^here^

#### 3.3. check_ptd_config()

In [ ]:
^fill^calc_point_defect_static.py(check_ptd_config)^here^

### 4. Run calculation function(s)

#### 4.1. Generate point defect system and evaluate the energy

In [ ]:
results_dict = pointdefect(lammps_command, system, potential, point_kwargs,
                           mpi_command = mpi_command,
                           etol = energytolerance,
                           ftol = forcetolerance,
                           maxiter = maxiterations,
                           maxeval = maxevaluations,
                           dmax = maxatommotion)

In [ ]:
results_dict.keys()

#### 4.2. Characterize if the defect has reconfigured

In [ ]:
cutoff = 1.05*ucell.box.a
results_dict.update(check_ptd_config(results_dict['system_ptd'], 
                                     point_kwargs, 
                                     cutoff))

In [ ]:
results_dict.keys()

### 5. Report results

#### 5.1. Define units for outputting values

- __length_unit__ is the unit of length to display relaxed lattice constants in.
- __energy_unit__ is the unit of energy to display cohesive energies in.

In [ ]:
length_unit = 'angstrom'
energy_unit = 'eV'

#### 5.2. Print $E_{coh}$ and $E_{ptd}^f$

In [ ]:
print('Ecoh =  ', uc.get_in_units(results_dict['E_coh'], energy_unit), energy_unit)
print('Eptd_f =', uc.get_in_units(results_dict['E_ptd_f'], energy_unit), energy_unit)

#### 5.3. Check configuration parameters

In [ ]:
print('Has the system (likely) reconfigured?', results_dict['has_reconfigured'])
if 'centrosummation' in results_dict:
    print('centrosummation =', uc.get_in_units(results_dict['centrosummation'], length_unit), length_unit)
if 'position_shift' in results_dict:
    print('position_shift = ', uc.get_in_units(results_dict['position_shift'], length_unit), length_unit)
if 'db_vect_shift' in results_dict:
    print('db_vect_shift =  ', uc.get_in_units(results_dict['db_vect_shift'], length_unit), length_unit)